In [21]:
import mibian
import datetime
import pandas as pd

def calc_implied_vol_delta_and_parity(row, underlying_price, days_to_expiry):
    interest_rate = 0.046
    delta_type = {'C': 'callDelta', 'P': 'putDelta', 'ATM Avg Put/Call': 'putDelta'}

    def calculate_mibian_bs(call_price, put_price, implied_vol=None):
        if implied_vol is None:
            print(f'Call Price: {call_price}, Put Price: {put_price}')
            c = mibian.BS([underlying_price, row['strike'], interest_rate, days_to_expiry], callPrice=call_price, putPrice=put_price)
            implied_vol = c.impliedVolatility
        c = mibian.BS([underlying_price, row['strike'], interest_rate, days_to_expiry], volatility=implied_vol)
        delta = getattr(c, delta_type[row['option_type']])
        return implied_vol, delta

    call_price, put_price = (row['mid_quote'], None) if row['option_type'] == 'C' else (None, row['mid_quote'])
    implied_vol, delta = calculate_mibian_bs(call_price, put_price)


    return pd.Series([implied_vol, delta])

expiry_date = '20230324'
trade_date = '2023-02-07'

days_to_expiry = (datetime.datetime.strptime(expiry_date, "%Y%m%d") - datetime.datetime.strptime(trade_date, "%Y-%m-%d")).days
working_dataset = f'./result-set/{trade_date}/P&L/dataset_{expiry_date}.csv' #_complete_
new_working_dataset = f'./result-set/{trade_date}/P&L/Delta-25&Above/dataset_{expiry_date}.csv' #_complete_
df = pd.read_csv(working_dataset)
forward = df[df['option_type'] == 'ATM Avg Put/Call']['strike'].values[0]

df[['Implied Vol', 'delta']] = df.apply(calc_implied_vol_delta_and_parity, args=(forward, days_to_expiry), axis=1)

filtered_df = df[df['delta'].abs() > 0.24]

# Calculate the total quantity from the removed rows
removed_quantity = df[df['delta'].abs() <= 0.24]['quantity'].sum()

# Calculate the number of rows in the filtered DataFrame
num_rows = len(filtered_df)

# Calculate the quantity to be added to each row in the filtered DataFrame
quantity_to_add = removed_quantity / num_rows

# Add the calculated quantity to the 'quantity' column of the filtered DataFrame
filtered_df['quantity'] += quantity_to_add

df.to_csv(working_dataset, index=False)
filtered_df.to_csv(new_working_dataset, index=False)



Call Price: None, Put Price: 0.1
Call Price: None, Put Price: 0.2
Call Price: None, Put Price: 0.3
Call Price: None, Put Price: 0.45
Call Price: None, Put Price: 0.675
Call Price: None, Put Price: 0.8999999999999999
Call Price: None, Put Price: 1.125
Call Price: None, Put Price: 1.4
Call Price: None, Put Price: 1.775
Call Price: None, Put Price: 1.975
Call Price: None, Put Price: 2.225
Call Price: None, Put Price: 2.5250000000000004
Call Price: None, Put Price: 2.85
Call Price: None, Put Price: 3.2
Call Price: None, Put Price: 3.7
Call Price: None, Put Price: 4.25
Call Price: None, Put Price: 4.85
Call Price: None, Put Price: 5.699999999999999
Call Price: None, Put Price: 6.699999999999999
Call Price: None, Put Price: 7.949999999999999
Call Price: None, Put Price: 9.55
Call Price: None, Put Price: 11.55
Call Price: None, Put Price: 14.05
Call Price: None, Put Price: 15.55
Call Price: None, Put Price: 17.2
Call Price: None, Put Price: 19.05
Call Price: None, Put Price: 20.25
Call Price:

C:\Users\USER\AppData\Local\Temp\ipykernel_15048\411795718.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['quantity'] += quantity_to_add
